In [1]:
//#r "./../../../../../public/src/L4-application/BoSSSpad/bin/Release/net6.0/BoSSSpad.dll"
#r "C:\Users\toprak\Documents\BoSSS-code\public\src\L4-application\BoSSSpad\bin\Release\net6.0\BoSSSpad.dll"
using System;
using System.Threading;
using System.Diagnostics;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.BoSSSpad;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Control;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.NSECommon;
//using BoSSS.XNSE_Solver;
using BoSSS.Solution.LoadBalancing;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.Utils;
using System.IO;

using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [2]:
// Is used at Jenkins to generate individual names (for output .json)
string dbname = System.Environment.GetEnvironmentVariable("DATABASE_NAME");
string buildname = System.Environment.GetEnvironmentVariable("JOB_NAME");
//defaults
buildname = String.IsNullOrEmpty(buildname)? "RotTorus3D" : buildname;
//string thedate = $"{System.DateTime.Today.Day}-{System.DateTime.Today.Month}-{System.DateTime.Today.Year}";
dbname = String.IsNullOrEmpty(dbname)? "condStudy" : dbname;
string table_name = String.Concat(buildname, "_", dbname);

In [3]:
// set parameterz
int[] PolyDegS = new int[] {2,3};
int[] ResArray = new int[] {16};
int[] core_sweep = { 4};
int AMRLevel = 1;
int MemoryPerCore = 3800;
bool submitJob = true;

In [4]:

//bool AMR = true;
//bool useLoadBal = true;
int NoOfTimeSteps = 25;
bool Steady = false;
bool IncludeConvection = true;
var Gshape = Shape.Torus;

In [5]:
ExecutionQueues

index,type,value
0,BoSSS.Application.BoSSSpad.SlurmClient,"SlurmClient Lb1 : mt64qocy@lcluster15.hrz.tu-darmstadt.de, Slurm account: special00006"
1,BoSSS.Application.BoSSSpad.SlurmClient,"SlurmClient Lb1 : mt64qocy@lcluster14.hrz.tu-darmstadt.de, Slurm account: special00006"
2,BoSSS.Application.BoSSSpad.MsHPC2012Client,"MS HPC client MSHPC-Gitrunner-DefaultTest @DC2, @\\dc1\userspace\toprak\BoSSS-Cluster4"
3,BoSSS.Application.BoSSSpad.MsHPC2012Client,"MS HPC client MSHPC-Gitrunner-DefaultTest @DC2, @\\dc1\userspace\toprak\BoSSS-Cluster6"
4,BoSSS.Application.BoSSSpad.MsHPC2012Client,"MS HPC client MSHPC-Gitrunner-DefaultTest @DC2, @\\dc1\userspace\toprak\BoSSS-Cluster"
5,BoSSS.Application.BoSSSpad.MsHPC2012Client,"MS HPC client MSHPC-Gitrunner-DefaultTest @DC2, @\\dc1\userspace\toprak\BoSSS-Cluster3D"


In [6]:
var myBatch = ExecutionQueues[1];
//var AddSbatchCmds = new List<string>();
//AddSbatchCmds.AddRange(new string[]{"#SBATCH -N 6", "#SBATCH -C avx512", "#SBATCH --mem-per-cpu="+MemoryPerCore});
//myBatch.AdditionalBatchCommands = AddSbatchCmds.ToArray();
//myBatch.AdditionalBatchCommands

In [7]:
string WFlowName = table_name;
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.Init(WFlowName,myBatch);
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
var myDB = BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.DefaultDatabase; 
myDB

Project name is set to 'RotTorus3D_validation'.
Creating database 'W:\work\scratch\mt64qocy\RotTorus3D_validation'.


{ Session Count = 0; Grid Count = 0; Path = W:\work\scratch\mt64qocy\RotTorus3D_validation }

In [8]:
static class Utils {
    // DOF per cell in 3D
    public static int Np(int p) {
        return (p*p*p + 6*p*p + 11*p + 6)/6;
    }    
    public static void ListTotalDOF(int[] PolyDegS,int[] ResArray) {
        foreach(int degree in PolyDegS){
                  Console.WriteLine($"For degree of {degree}, DOF per cell: {Np(degree)}");
            foreach(int res in ResArray)
                Console.WriteLine($"\u2022 Res: {res}: {Np(degree)*Math.Pow(res,3)} ");
                }
    }  
}
    Utils.ListTotalDOF(PolyDegS,ResArray);

For degree of 2, DOF per cell: 10
• Res: 16: 40960 
For degree of 3, DOF per cell: 20
• Res: 16: 81920 


using 

In [9]:
int SpaceDim = 3;
double _partRad = 0.3900;

In [10]:
double xMax = 1.0, yMax = 1.0, zMax = 1.0;
double xMin = -1.0, yMin = -1.0,zMin = -1.0;

In [11]:
//var gridList = myDB.Grids;
//var aa = gridList[0];
//IGrid grid = myDB.Controller.DBDriver.LoadGrid(aa.ID,myDB);
//GridCommons abba = (GridCommons)grid;


In [12]:
List<IGridInfo> Grids = new List<IGridInfo>();
if (SpaceDim ==2){
foreach(var Res in ResArray){

    var _xNodes = GenericBlas.Linspace(xMin, xMax, Res + 1);
    var _yNodes = GenericBlas.Linspace(yMin, yMax, Res + 1);

    GridCommons grd;
    string gname = "RotTorus2D_1BCs";

    Console.WriteLine("Looking for {0} with {1} cells",gname,(_xNodes.Length-1)*(_yNodes.Length-1));

    var tmp = new List<IGridInfo>();

    foreach(var grid in myDB.Grids){
        try{
            if (String.IsNullOrEmpty(grid.Name))
                continue;
                //grid.Name = "Null";
            Console.WriteLine("Found {0} with {1} cells on the database",grid.Name,grid.NumberOfCells);
            bool IsMatch = grid.Name.Equals(gname)&&grid.NumberOfCells==(_xNodes.Length-1)*(_yNodes.Length-1);
            if(IsMatch) tmp.Add(grid);
        }
        catch(Exception ex) {
            Console.WriteLine(ex.Message);
        }
    }
    Console.WriteLine("Found {0} times grids with the same configuration for the Res={1}",tmp.Count,Res);
    
    if(tmp.Count() >= 1){
        Console.WriteLine("Chosing the first one found: "+tmp.Pick(0).Name);
        Grids.Add(tmp.Pick(0));
        //IGridInfo firstPick = Grids.ToArray()[0];
        //var ImportedGrid = myDB.Controller.DBDriver.LoadGrid(firstPick.ID,myDB);
        //grd = (GridCommons)ImportedGrid;EdgeTagName 
        //Console.WriteLine("Chosen Grid:{0}",ImportedGrid.ToString());
        continue;

    }else{

        grd = Grid2D.Cartesian2DGrid(_xNodes, _yNodes);

    
        grd.Name = gname;

        grd.EdgeTagNames.Add(1, "Pressure_Outlet");
        grd.DefineEdgeTags(delegate (double[] X) {
            return 1;
        });
                
    myDB.SaveGrid(ref grd);
    Grids.Add(grd);
    }
    
}
Console.WriteLine("\nThe grids to be simulated:");
}
Grids


In [13]:
if (SpaceDim ==3){
foreach(var Res in ResArray){

    var _xNodes = GenericBlas.Linspace(xMin, xMax, Res + 1);
    var _yNodes = GenericBlas.Linspace(yMin, yMax, Res + 1);
    var _zNodes = GenericBlas.Linspace(zMin, zMax, Res + 1);

    GridCommons grd;
    string gname = "RotTorus3D_1BCs";

    Console.WriteLine("Looking for {0} with {1} cells",gname,(_xNodes.Length-1)*(_yNodes.Length-1)*(_zNodes.Length-1));

    var tmp = new List<IGridInfo>();

    foreach(var grid in myDB.Grids){
        try{
            if (String.IsNullOrEmpty(grid.Name))
                continue;
                //grid.Name = "Null";
            Console.WriteLine("Found {0} with {1} cells on the database",grid.Name,grid.NumberOfCells);
            bool IsMatch = grid.Name.Equals(gname)&&grid.NumberOfCells==(_xNodes.Length-1)*(_yNodes.Length-1)*(_zNodes.Length-1);
            if(IsMatch) tmp.Add(grid);
        }
        catch(Exception ex) {
            Console.WriteLine(ex.Message);
        }
    }
    Console.WriteLine("Found {0} times grids with the same configuration for the Res={1}",tmp.Count,Res);
    
    if(tmp.Count() >= 1){
        Console.WriteLine("Chosing the first one found: "+tmp.Pick(0).Name);
        Grids.Add(tmp.Pick(0));
        //IGridInfo firstPick = Grids.ToArray()[0];
        //var ImportedGrid = myDB.Controller.DBDriver.LoadGrid(firstPick.ID,myDB);
        //grd = (GridCommons)ImportedGrid;EdgeTagName 
        //Console.WriteLine("Chosen Grid:{0}",ImportedGrid.ToString());
        continue;

    }else{

        grd = Grid3D.Cartesian3DGrid(_xNodes, _yNodes,_zNodes);   
        grd.Name = gname;

        grd.EdgeTagNames.Add(1, "Pressure_Outlet");
        grd.DefineEdgeTags(delegate (double[] X) {
            return 1;
        });
                
    myDB.SaveGrid(ref grd);
    Grids.Add(grd);
    }
    
}
Console.WriteLine("\nThe grids to be simulated:");
}
Grids


Looking for RotTorus3D_1BCs with 4096 cells
Found 0 times grids with the same configuration for the Res=16
Grid Edge Tags changed.

The grids to be simulated:


#0: { Guid = e830c015-5cec-4ed4-83dc-6cb486f5f7bb; Name = RotTorus3D_1BCs; Cell Count = 4096; Dim = 3 }


double xMax = 1.0, yMax = 1.0, zMax=1.0;
double xMin = -1.0, yMin = -1.0,zMin = -1.0;

In [14]:
_partRad/(2/(double)ResArray[0]/4)

12.48

## Generate Control object

### governing equations
- incompressible transient Navier-Stokes:
<br>$\frac{\partial \rho \vec{u}}{\partial t}+ \nabla \cdot ( \rho \vec{u} \otimes \vec{u}) + \nabla p - \eta \Delta \vec{u} = \vec{f} \ \ in \ \ \Omega_F(t) \times (0,T)$
<br>$\nabla \cdot \vec{u} = 0 \quad in \ \ \Omega_F(t) \times (0,T)$
- with inital conditions:
<br>$\vec{u}(\vec{x},0)  =  \vec{0} \quad on \ \ \Omega_F(0)$
<br>$p(\vec{x},0)  =  0 \quad  on \ \ \Omega_F(0)$
- with boundary conditions:
<br>$\vec{u}(\vec{x},t)  =  \vec{u}_{Inlet} \ \  on \ \ \Gamma_{Inlet} = \{ \vec{X} \in \partial \Omega_F(t) |  x=-2 \}$
<br>$p \mathbf{I} - \frac{1}{Re} \nabla \vec{u} \vec{n}_{ \Gamma_{pOut} } = 0 \ \ on \ \ \Gamma_{pOut} = \partial \Omega \backslash \Gamma_{Inlet} $ 
<br>$\vec{u}(\vec{x},t) = \boldsymbol{\omega}(t) \times \vec{r} \quad on \ \ \mathcal{J} = \partial \Omega_S \cap \partial \Omega_F$

- Inlet-Velocity $u_{Inlet}=\frac{Re*\mu_A}{\rho_A*d_{hyd}}$
- angular velocity of rotating sphere $\boldsymbol{\omega}(t)=\frac{Re*\mu_A}{\rho_A*d_{hyd}*1m}$

In [15]:
Func<IGridInfo, int, double, double, string, bool, bool, double, XNSE_Control> GenXNSECtrl = delegate(IGridInfo grd, int k, double Viscosity, double tiltAngle, string RotAxis, bool AMR, bool LoadBalance, double AggThrs){
    XNSE_Control C = new XNSE_Control();

    // basic database options
    // ======================
    C.SetDatabase(myDB);
    C.savetodb = true;
    int J  = grd.NumberOfCells;
    string rotAxis;

    // Physical Parameters
    // ===================
    const double rhoA = 1;
    const double Re = 1000;
    double muA = Viscosity;
    
    double partRad = _partRad;
    double d_hyd = 2*partRad;
    double VelocityMax = Re * muA / rhoA / d_hyd;
    double anglev = VelocityMax / partRad;
    double[] pos = new double[SpaceDim];
    double ts = (Math.PI/2)/anglev/NoOfTimeSteps; // until symmetry (pi/2)   double ts = Math.PI/anglev/NoOfTimeSteps/10;
    
    C.PhysicalParameters.IncludeConvection = IncludeConvection;
    C.PhysicalParameters.Material = true;
    C.PhysicalParameters.rho_A = rhoA;
    C.PhysicalParameters.mu_A = muA;


    if (RotAxis == null)
        rotAxis = "y";
    else
        rotAxis = RotAxis;

    C.SessionName = string.Format("J{0}_k{1}_Re{2}_agg{3}_ti{4:f3}at{5}_r{6}_AMR{7}_LB{8}", J, k,Re,AggThrs,tiltAngle,rotAxis.ToUpper(),_partRad,AMR,LoadBalance);
    if(IncludeConvection){
        C.SessionName += "_NSE";
        C.Tags.Add("NSE");
    } else {
        C.SessionName += "_Stokes";
        C.Tags.Add("Stokes");
    }
    C.Tags.Add(SpaceDim + "D");
    // if(Steady)C.Tags.Add("steady");
    // else C.Tags.Add("transient");

    // DG degrees
    // ==========
    C.SetFieldOptions(k, Math.Max(k, 2));
    C.saveperiod = 10;
    //C.TracingNamespaces = "*";

    C.GridGuid = grd.ID;
    C.GridPartType = GridPartType.Hilbert;
    //C.DynamicLoadbalancing_ClassifierType = ClassifierType.CutCells;
    C.DynamicLoadBalancing_On = LoadBalance;
    C.DynamicLoadBalancing_RedistributeAtStartup = true;
    C.DynamicLoadBalancing_Period = 10;
    C.DynamicLoadBalancing_ImbalanceThreshold = 0.1;
    //C.DynamicLoadBalancing_CellCostEstimatorFactories = BoSSS.Application.XNSE_Solver.Loadbalancing.XNSECellCostEstimator.Factory().ToList();

    C.ImmediatePlotPeriod = -1;
    C.SuperSampling = 0;
    
    double rateOfRadius = 0.0; //-1/(double)NoOfTimeSteps/ts;
    double ringRad = partRad/1.5;
    C.Rigidbody.SetParameters(pos,anglev,partRad,SpaceDim,ringRad,rateOfRadius: rateOfRadius);
    C.SessionName += $"_dRdt{rateOfRadius:f2}";
    C.Rigidbody.SpecifyShape(Gshape);
    C.Rigidbody.SetRotationAxis(rotAxis);
    var tiltAxis = new Vector(1,0,0);
    //Console.WriteLine($"tiltAxis.Dim: {tiltAxis.Dim}");
    Console.WriteLine("Angular Velocity: {0}",anglev);
    C.SessionName += $"_W{anglev:f2}";

    C.Rigidbody.SetTilt(tiltAxis, tiltAngle);
    C.AddInitialValue(VariableNames.LevelSetCGidx(0), new Formula("X => -1"));
    C.UseImmersedBoundary = true;
    
    C.AddInitialValue("Pressure", new Formula(@"X => 0"));
    C.AddBoundaryValue("Pressure_Outlet");
    //double inletdelay = 5*ts;
    //C.AddBoundaryValue("Velocity_inlet","VelocityX",new Formula($"(X,t) => {VelocityIn}*(double)(t<={inletdelay}?(t/{inletdelay}):1)",true));
    //C.AddBoundaryValue("Velocity_inlet","VelocityX",new Formula($"(X) => {VelocityIn}"));
    //C.AddInitialValue("VelocityX", new Formula($"(X) => {VelocityIn}"));

    C.CutCellQuadratureType = BoSSS.Foundation.XDG.XQuadFactoryHelper.MomentFittingVariants.Saye;
    C.UseSchurBlockPrec = true;
    C.AgglomerationThreshold = AggThrs;
    C.AdvancedDiscretizationOptions.ViscosityMode = ViscosityMode.FullySymmetric;
    C.Option_LevelSetEvolution2 = LevelSetEvolution.Prescribed;
    C.Option_LevelSetEvolution = LevelSetEvolution.None;
    C.Timestepper_LevelSetHandling = LevelSetHandling.LieSplitting;
    C.LinearSolver = new BoSSS.Solution.AdvancedSolvers.OrthoMGSchwarzConfig() {
        NoOfMultigridLevels = 5,
        ConvergenceCriterion = 1E-3,
        MaxSolverIterations = 100,
        //MaxKrylovDim = 30,
        TargetBlockSize = 10000,
        //verbose = true
    };

    C.LinearSolver = LinearSolverCode.exp_Kcycle_schwarz.GetConfig();
    C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    C.NonLinearSolver.ConvergenceCriterion = 1E-3;
    C.NonLinearSolver.MaxSolverIterations = 50;
    C.NonLinearSolver.verbose = true;

    C.AdaptiveMeshRefinement = AMR;
    if (AMR) {
        C.SetMaximalRefinementLevel(AMRLevel);
        C.AMR_startUpSweeps = AMRLevel;
    }

    // Timestepping
    // ============
    double dt = -1;
    if(Steady){
        C.TimesteppingMode = AppControl._TimesteppingMode.Steady;
        dt = 1000;
        C.NoOfTimesteps = 1;
    } else {
        C.TimesteppingMode = AppControl._TimesteppingMode.Transient;        
        dt = ts;        
        C.NoOfTimesteps = NoOfTimeSteps;
    }
    C.TimeSteppingScheme = TimeSteppingScheme.ImplicitEuler; //BD4
    C.dtFixed = dt;
    C.SkipSolveAndEvaluateResidual = false;
    C.SessionName = "Solver" + !C.SkipSolveAndEvaluateResidual + "_" + C.SessionName;
    var config = new OperatorAnalysisConfig();
    config.CalculateMassMatrix = true;
    config.CalculateGlobalConditionNumbers = false;
    config.CalculateStencilConditionNumbers = true;
    config.PlotStencilConditionNumbers = true;

    C.PostprocessingModules.Add(new BoSSS.Application.XNSE_Solver.Logging.CondLogger(config));
            
    C.SessionName += "_notebookTest"; 

    return C;
};

In [16]:
GenXNSECtrl(Grids[0],1,1,0.0,"z",false,false,0)

Angular Velocity: 3287.310979618672


UseImmersedBoundary,True
TimesteppingMode,Transient
Timestepper_LevelSetHandling,LieSplitting
ControlFileText,<null>
GeneratedFromCode,False
ControlFileText_Index,0
NoOfMultigridLevels,1000000
FieldOptions,"[ [Velocity*, BoSSS.Solution.Control.FieldOpts], [Pressure, BoSSS.Solution.Control.FieldOpts], [PhiDG, BoSSS.Solution.Control.FieldOpts], [Phi, BoSSS.Solution.Control.FieldOpts], [Phi2DG, BoSSS.Solution.Control.FieldOpts], [Phi2, BoSSS.Solution.Control.FieldOpts], [Curvature, BoSSS.Solution.Control.FieldOpts] ]"
BoundaryValues,"[ [Pressure_Outlet, BoSSS.Solution.Control.AppControl+BoundaryValueCollection] ]"
BoundaryValueChanges,[ ]
InitialValues_EvaluatorsVec,"[ [Phi, BoSSS.Foundation.ScalarFunctionTimeDep], [Pressure, BoSSS.Foundation.ScalarFunctionTimeDep] ]"


In [17]:
var ViscositySweep = new double[]{1E-2};
var tiltAngles = new double[]{Math.PI/4,Math.PI/5,Math.PI/6,Math.PI/3}; //Math.PI/4
var axes = new string[]{"y"};
var AMRarray = new bool[] {false};
var LoadBalancearray = new bool[] {false};
var aggThresolds = new double[] { 0.5};

List<XNSE_Control> controls = new List<XNSE_Control>();
foreach(IGridInfo grd in Grids){
    foreach(int k in PolyDegS){
        foreach(double v in ViscositySweep)
            foreach(double tiltAngle in tiltAngles)
                foreach(string axis in axes)
                    foreach(bool AMR in AMRarray)
                        foreach(bool LoadBalance in LoadBalancearray)
                            foreach(double agg in aggThresolds)
                               controls.Add(GenXNSECtrl(grd,k,v,tiltAngle,axis,AMR,LoadBalance,agg));
    }
} controls.Select(s=>s.dtFixed)

Angular Velocity: 32.87310979618672
Angular Velocity: 32.87310979618672
Angular Velocity: 32.87310979618672
Angular Velocity: 32.87310979618672
Angular Velocity: 32.87310979618672
Angular Velocity: 32.87310979618672
Angular Velocity: 32.87310979618672
Angular Velocity: 32.87310979618672


[ 0.0019113449704440302, 0.0019113449704440302, 0.0019113449704440302, 0.0019113449704440302, 0.0019113449704440302, 0.0019113449704440302, 0.0019113449704440302, 0.0019113449704440302 ]

In [18]:
controls.Select(s=>s.SessionName)

[ SolverFalse_J4096_k2_Re1000_agg0.5_ti0.785atY_r0.39_AMRFalse_LBFalse_NSE_dRdt0.00_W32.87validation0.5, SolverFalse_J4096_k2_Re1000_agg0.5_ti0.628atY_r0.39_AMRFalse_LBFalse_NSE_dRdt0.00_W32.87validation0.5, SolverFalse_J4096_k2_Re1000_agg0.5_ti0.524atY_r0.39_AMRFalse_LBFalse_NSE_dRdt0.00_W32.87validation0.5, SolverFalse_J4096_k2_Re1000_agg0.5_ti1.047atY_r0.39_AMRFalse_LBFalse_NSE_dRdt0.00_W32.87validation0.5, SolverFalse_J4096_k3_Re1000_agg0.5_ti0.785atY_r0.39_AMRFalse_LBFalse_NSE_dRdt0.00_W32.87validation0.5, SolverFalse_J4096_k3_Re1000_agg0.5_ti0.628atY_r0.39_AMRFalse_LBFalse_NSE_dRdt0.00_W32.87validation0.5, SolverFalse_J4096_k3_Re1000_agg0.5_ti0.524atY_r0.39_AMRFalse_LBFalse_NSE_dRdt0.00_W32.87validation0.5, SolverFalse_J4096_k3_Re1000_agg0.5_ti1.047atY_r0.39_AMRFalse_LBFalse_NSE_dRdt0.00_W32.87validation0.5 ]

In [19]:
int k=1;
if (submitJob){
    foreach(var ctrl in controls){
        string sessname = ctrl.SessionName;
        foreach(int cores in core_sweep){
            ctrl.SessionName = sessname + "_ct"+cores; //+"_mue_"+ctrl.PhysicalParameters.mu_A;
                    var aJob   = new Job("Rot"+Gshape+"_"+ctrl.SessionName,typeof(XNSE));
            //Console.WriteLine(ctrl.Rigidbody.GetTilt());
            aJob.SetControlObject(ctrl);
            aJob.NumberOfMPIProcs         = cores;
            aJob.ExecutionTime            = "71:59:00";
            aJob.UseComputeNodesExclusive = true;

            //List<string> Cmdtmp = myBatch.AdditionalBatchCommands.ToList();
            //int NoOfNodes = 1;
            //Cmdtmp.Add($"#SBATCH -N {NoOfNodes}");
            //myBatch.AdditionalBatchCommands = Cmdtmp.ToArray();

            aJob.Activate(myBatch);
            Console.WriteLine("Submitted {0}th",k) ;
            k++;           
        }       
    }
}

Deployments so far (0): ;
Success: 0
job submit count: 0


unable to determine job status - unknown


Deploying job RotTorus_SolverFalse_J4096_k2_Re1000_agg0.5_ti0.785atY_r0.39_AMRFalse_LBFalse_NSE_dRdt0.00_W32.87validation0.5_ct4 ... 
Deploying executables and additional files ...
Deployment directory: W:\work\scratch\mt64qocy\RotTorus3D_validation-XNSE_Solver2024Jan22_154608.032569
copied 43 files.
   written file: control.obj
   copied 'linux/amd64-openmpi' runtime.
deployment finished.
Instance #1 of ssh client mt64qocy@lcluster14.hrz.tu-darmstadt.de instantiated.

Submitted 1th
Deployments so far (0): ;
Success: 0
job submit count: 0


unable to determine job status - unknown


Deploying job RotTorus_SolverFalse_J4096_k2_Re1000_agg0.5_ti0.628atY_r0.39_AMRFalse_LBFalse_NSE_dRdt0.00_W32.87validation0.5_ct4 ... 
Deploying executables and additional files ...
Deployment directory: W:\work\scratch\mt64qocy\RotTorus3D_validation-XNSE_Solver2024Jan22_154617.879596
copied 43 files.
   written file: control.obj
   copied 'linux/amd64-openmpi' runtime.
deployment finished.

Submitted 2th
Deployments so far (0): ;
Success: 0
job submit count: 0


...

Deploying job RotTorus_SolverFalse_J4096_k3_Re1000_agg0.5_ti0.628atY_r0.39_AMRFalse_LBFalse_NSE_dRdt0.00_W32.87validation0.5_ct4 ... 
Deploying executables and additional files ...
Deployment directory: W:\work\scratch\mt64qocy\RotTorus3D_validation-XNSE_Solver2024Jan22_154635.098188
copied 43 files.
   written file: control.obj
   copied 'linux/amd64-openmpi' runtime.
deployment finished.

Submitted 6th
Deployments so far (0): ;
Success: 0
job submit count: 0


unable to determine job status - unknown


Deploying job RotTorus_SolverFalse_J4096_k3_Re1000_agg0.5_ti0.524atY_r0.39_AMRFalse_LBFalse_NSE_dRdt0.00_W32.87validation0.5_ct4 ... 
Deploying executables and additional files ...
Deployment directory: W:\work\scratch\mt64qocy\RotTorus3D_validation-XNSE_Solver2024Jan22_154639.477838
copied 43 files.
   written file: control.obj
   copied 'linux/amd64-openmpi' runtime.
deployment finished.

Submitted 7th
Deployments so far (0): ;
Success: 0
job submit count: 0


unable to determine job status - unknown


Deploying job RotTorus_SolverFalse_J4096_k3_Re1000_agg0.5_ti1.047atY_r0.39_AMRFalse_LBFalse_NSE_dRdt0.00_W32.87validation0.5_ct4 ... 
Deploying executables and additional files ...
Deployment directory: W:\work\scratch\mt64qocy\RotTorus3D_validation-XNSE_Solver2024Jan22_154644.058965
copied 43 files.
   written file: control.obj
   copied 'linux/amd64-openmpi' runtime.
deployment finished.

Submitted 8th


In [20]:
wmg.DefaultDatabase.Sessions

#0: RotTorus3D_validation	SolverFalse_J4096_k2_Re1000_agg0.5_ti0.785atY_r0.39_AMRFalse_LBFalse_NSE_dRdt0.00_W32.87validation0.5_ct4*	1/22/2024 3:46:43 PM	e95a66f0...


In [21]:
//j.AllDependentAssemblies.Select(ass => ass.Location)